# Define imports

In [2]:
#!pip install opencv-python
import os
import cv2
import os
import glob
import numpy as np
from Regressor_Model_Controller import Regressor_Model_Controller
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
import torch
import torchvision.transforms as transforms
import math
import random
from CustomData import ImageDataset
from utils import Utils


torch.set_default_tensor_type(torch.FloatTensor)

# Define constants

In [3]:
DIPALI_HOME = './DLWG/DataSet/*.jpg'
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
REGRESSOR_SAVED_MODEL_PATH= './Model/Regressor/Regressor.pth'
REGRESSOR_EPOCH = 200
REGRESSOR_LOSS_PLOT_PATH = "./Plots/Regressor/Regressor_Loss_plot"
REGRESSOR_LR = 0.0001
REGRESSOR_WEIGHT_DECAY = 1e-5
REGRESSOR_IN_CHANNEL = 1
REGRESSOR_HIDDEN_CHANNEL = 3
REGRESSOR_OUT_DIMS = 2
REGRESSOR_BATCH_SIZE_CPU = 32
REGRESSOR_BATCH_SIZE_CUDA = 16
REGRESSOR_TEST_SPLIT=0.1

os.makedirs("./Model/Regressor/", exist_ok=True)
os.makedirs("./Plots/Regressor/",exist_ok=True)

# Use GPU if available

In [4]:
def get_device():
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        is_cuda_present = True if torch.cuda.is_available() else False
        num_workers = 8 if is_cuda_present else 0

        return device, is_cuda_present, num_workers

# load augmented image data

In [5]:
train_data_set_dir = './TrainSet'
test_data_set_dir = './TestSet'

train_image_paths = os.listdir(train_data_set_dir)
test_image_paths = os.listdir(test_data_set_dir)

train_image_paths = [image_path for image_path in train_image_paths if image_path.endswith('.jpg')]
test_image_paths = [image_path for image_path in test_image_paths if image_path.endswith('.jpg')]

random.shuffle(train_image_paths)
random.shuffle(test_image_paths)

train_image_data = [cv2.imread(os.path.join(train_data_set_dir, image_path)) for image_path in train_image_paths]
test_image_data = [cv2.imread(os.path.join(test_data_set_dir, image_path)) for image_path in test_image_paths]


# Images to  L* a* b* color space convertion

In [ ]:
# LAB color space
train_image_data = [cv2.cvtColor(image, cv2.COLOR_BGR2LAB) for image in train_image_data]
test_image_data = [cv2.cvtColor(image, cv2.COLOR_BGR2LAB) for image in test_image_data]


### Display L* a* b images

In [ ]:
def showImageLab(image_Lab):
    #image_bgr = cv2.imread(image_file)
    #image_Lab = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2LAB)

    L = image_Lab[:, :, 0]
    a = image_Lab[:, :, 1]
    b = image_Lab[:, :, 2]
    print(L)

    plt.subplot(1, 3, 1)
    plt.title('L')
    plt.gray()
    plt.imshow(L)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title('a')
    plt.gray()
    plt.imshow(a)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title('b')
    plt.gray()
    plt.imshow(b)
    plt.axis('off')

    plt.show() 
    
#print(test_image_data[:5])

showImageLab(train_image_data[2])


In [ ]:
# Convert images to tensors
transform_color = transforms.Compose([transforms.ToTensor()])

transformed_train_image_data = []
transformed_test_image_data = []

for image in train_image_data:
    # train image to tensor
    image_lab = transform_color(image)
    image_gray = image_lab[0:1,:,:]
    image_a = image_lab[1:2, :, :]
    image_b = image_lab[2:3, :, :]
    transformed_train_image_data.append((image_gray, image_a, image_b))

for image in test_image_data:
    # test image to tensor
    image_lab = transform_color(image)
    image_gray = image_lab[0:1,:,:]
    print(image_gray.shape)
    image_a = image_lab[1:2, :, :]
    image_b = image_lab[2:3, :, :]
    transformed_test_image_data.append((image_gray, image_a, image_b))


In [6]:
transform = transforms.Compose([transforms.ToTensor()])

transformed_train_dataset = ImageDataset(root_dir='./TrainSet',
                                         transform=transform,
                                         regressor_only=True)
transformed_test_dataset = ImageDataset(root_dir='./TestSet',
                                        transform=transform,
                                        regressor_only=True)
print(transformed_test_dataset[0])

(tensor([[[0.0667, 0.0667, 0.0706,  ..., 0.1176, 0.1490, 0.1647],
         [0.0706, 0.0745, 0.0745,  ..., 0.1176, 0.1490, 0.1725],
         [0.0824, 0.0902, 0.0902,  ..., 0.1176, 0.1490, 0.1765],
         ...,
         [0.3059, 0.4000, 0.3098,  ..., 0.2353, 0.0588, 0.4588],
         [0.3922, 0.4196, 0.3059,  ..., 0.1804, 0.0980, 0.5255],
         [0.4784, 0.3843, 0.3255,  ..., 0.0824, 0.2902, 0.6824]]]), tensor([[[0.5098, 0.5098, 0.5098,  ..., 0.5176, 0.5255, 0.5216],
         [0.5098, 0.5098, 0.5098,  ..., 0.5176, 0.5216, 0.5216],
         [0.5020, 0.5098, 0.5098,  ..., 0.5176, 0.5216, 0.5216],
         ...,
         [0.5294, 0.5294, 0.5333,  ..., 0.5373, 0.5412, 0.5176],
         [0.5294, 0.5294, 0.5294,  ..., 0.5412, 0.5373, 0.5176],
         [0.5294, 0.5294, 0.5294,  ..., 0.5490, 0.5255, 0.5137]]]), tensor([[[0.4902, 0.4902, 0.4902,  ..., 0.5137, 0.5137, 0.5216],
         [0.4902, 0.4902, 0.4902,  ..., 0.5137, 0.5216, 0.5216],
         [0.4902, 0.4902, 0.4902,  ..., 0.5137, 0.5216,

# Train/Test batch data loading

In [7]:
train_image_data = []
test_image_data = []

batch_size = REGRESSOR_BATCH_SIZE_CUDA
# include both color and gray images
trainloader = torch.utils.data.DataLoader(transformed_train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)
testloader = torch.utils.data.DataLoader(transformed_test_dataset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

# Train the Regressor

In [8]:
def train_regressor(augmented_dataset_batch, device):
        regressor_train_arguments = {
            "data_loader": augmented_dataset_batch,
            "saved_model_path": REGRESSOR_SAVED_MODEL_PATH,
            "epochs": REGRESSOR_EPOCH,
            "learning_rate": REGRESSOR_LR,
            "weight_decay": REGRESSOR_WEIGHT_DECAY,
            "in_channel": REGRESSOR_IN_CHANNEL,
            "hidden_channel": REGRESSOR_HIDDEN_CHANNEL,
            "out_dims": REGRESSOR_OUT_DIMS,
            "loss_plot_path": REGRESSOR_LOSS_PLOT_PATH
        }

        regressor_manager = Regressor_Model_Controller()
        regressor_manager.train(regressor_train_arguments, device)

def test_regressor(augmented_dataset_batch, device):
        regressor_arguments = {
            "data_loader": augmented_dataset_batch,
            "saved_model_path": REGRESSOR_SAVED_MODEL_PATH,
            "in_channel": REGRESSOR_IN_CHANNEL,
            "hidden_channel": REGRESSOR_HIDDEN_CHANNEL,
            "out_dims": REGRESSOR_OUT_DIMS,
            "loss_plot_path": REGRESSOR_LOSS_PLOT_PATH
        }

        regressor_manager = Regressor_Model_Controller()
        regressor_manager.test(regressor_arguments, device)

In [9]:

device, is_cuda_present, num_workers = Utils.get_device()
train_regressor(trainloader,device)

..Regressor training started..
epoch: 0, loss: 0.948368490091525
epoch: 1, loss: 0.14347727694257628
epoch: 2, loss: 0.098536484692886
epoch: 3, loss: 0.07835714912653202
epoch: 4, loss: 0.06640737788620754
epoch: 5, loss: 0.061646172929613385
epoch: 6, loss: 0.05328369171184022
epoch: 7, loss: 0.052530278986523626
epoch: 8, loss: 0.04928778022076585
epoch: 9, loss: 0.04598277894911007
epoch: 10, loss: 0.04097573346007266
epoch: 11, loss: 0.04475397317764873
epoch: 12, loss: 0.03994504684669664
epoch: 13, loss: 0.040644261873239884
epoch: 14, loss: 0.037996210312485346
epoch: 15, loss: 0.036500815527688246
epoch: 16, loss: 0.040909150297011365
epoch: 17, loss: 0.03646674375522707
epoch: 18, loss: 0.039460703354052384
epoch: 28, loss: 0.02761747024123906
epoch: 29, loss: 0.028205218961375067
epoch: 30, loss: 0.028559753383888165
epoch: 31, loss: 0.02820867804257432
epoch: 32, loss: 0.02701571576471906
epoch: 33, loss: 0.025637541941250674
epoch: 34, loss: 0.027003439534382778
epoch: 35,

<Figure size 432x288 with 0 Axes>

In [ ]:
test_regressor(testloader,device)